In [ ]:

# Descomentar cuando se ejecuta desde la página web
import piplite
await piplite.install(['ipywidgets'])

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
#from ipywidgets import interact, FloatSlider
#from IPython.display import display

from ipywidgets import interact, FloatSlider, VBox, HBox, Button, Output
from IPython.display import display, clear_output

def campo_electrico(t):
    """Campo eléctrico E(t) = sin(2π t)"""
    return np.sin(2 * np.pi * t)

def ecuacion_movimiento(t, y):
    """
    Sistema de EDOs: y = [x, v]
    dx/dt = v
    dv/dt = -E(t)
    """
    x, v = y
    dxdt = v
    dvdt = -campo_electrico(t)
    return [dxdt, dvdt]

def calcular_trayectoria(t_i, duracion=1.5):
    """
    Integra la trayectoria desde t_i durante 1.5 ciclos
    Condiciones iniciales: x(t_i) = 0, v(t_i) = 0
    """
    t_span = (t_i, t_i + duracion)
    t_eval = np.linspace(t_i, t_i + duracion, 1000)
    y0 = [0, 0]  # [x(t_i), v(t_i)]
    
    sol = solve_ivp(ecuacion_movimiento, t_span, y0, t_eval=t_eval, 
                    method='RK45', dense_output=True)
    
    return sol.t, sol.y[0], sol.y[1]  # t, x(t), v(t)

def encontrar_recombinacion(t, x, v, t_i):
    """
    Encuentra el primer tiempo t_rec > t_i donde x cruza x=0
    """
    # Buscar cruces después de t_i (ignorar el punto inicial)
    indices = np.where(t > t_i + 0.01)[0]  # pequeño offset para evitar t_i
    
    for i in indices[:-1]:
        # Detectar cruce: cambio de signo en x
        if x[i] * x[i+1] < 0:
            # Interpolación lineal para mayor precisión
            t_rec = t[i] + (t[i+1] - t[i]) * (-x[i]) / (x[i+1] - x[i])
            v_rec = v[i] + (v[i+1] - v[i]) * (-x[i]) / (x[i+1] - x[i])
            return t_rec, v_rec
    
    return None, None  # No hay recombinación

def plot_trayectoria(t_i):
    """
    Visualiza la trayectoria y el campo eléctrico
    """
    # Calcular trayectoria
    t, x, v = calcular_trayectoria(t_i)
    
    # Encontrar recombinación
    t_rec, v_rec = encontrar_recombinacion(t, x, v, t_i)
    
    # Crear figura con dos ejes y
    fig, ax1 = plt.subplots(figsize=(10, 3))
    
    # Eje izquierdo: Trayectoria x(t)
    color_traj = 'tab:blue'
    ax1.set_xlabel('Tiempo (ciclos)', fontsize=12)
    ax1.set_xlim((0,2))
    ax1.set_ylim((-0.1,0.1))
    ax1.set_ylabel('Posición x(t)', color=color_traj, fontsize=12)
    # ax1.plot(t, x, color=color_traj, linewidth=2, label='Trayectoria x(t)')
    ax1.plot(t, x, color=color_traj, linewidth=2)
    ax1.axhline(y=0, color='black', linestyle='--', alpha=0.3)
    ax1.tick_params(axis='y', labelcolor=color_traj)
    ax1.grid(True, alpha=0.3)
    
    # Marcar punto de ionización
    #ax1.plot(t_i, 0, 'go', markersize=10, label=f'Ionización ($t_i$={t_i:.3f})')
    ax1.plot(t_i, 0, 'go', markersize=10)
    
    # Marcar recombinación si existe
    # if t_rec is not None:
    #     ax1.plot(t_rec, 0, 'ro', markersize=10, label=f'Recombinación ($t_{{rec}}$={t_rec:.3f})')
    
    # ax1.legend(loc='upper left')
    
    # Eje derecho: Campo eléctrico
    ax2 = ax1.twinx()
    color_campo = 'tab:red'
    ax2.set_ylabel('Campo E(t)', color=color_campo, fontsize=12)
    t_campo = np.linspace(0, 2, 500)
    E_campo = campo_electrico(t_campo)
    ax2.plot(t_campo, E_campo, color=color_campo, linewidth=1.5, 
             alpha=0.6, label='Campo E(t)')
    ax2.tick_params(axis='y', labelcolor=color_campo)
    
    # Título y cuadro de texto
    plt.title('Trayectoria del electrón en campo láser', fontsize=14, fontweight='bold')
    
    # Cuadro de texto con información
    if t_rec is not None:
        info_text = f'$t_i$ = {t_i:.4f}\n$t_{{rec}}$ = {t_rec:.4f}\n$v(t_{{rec}})$ = {v_rec:.4f}'
    else:
        info_text = f'$t_i$ = {t_i:.4f}\n$t_{{rec}}$ = No hay recombinación\n$v(t_{{rec}})$ = N/A'
    
    ax1.text(1.2, 0.98, info_text, transform=ax1.transAxes,
             fontsize=11, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

# # Widget interactivo
# interact(plot_trayectoria, 
#          t_i=FloatSlider(min=0.0, max=1.0, step=0.01, value=0, 
#                         description='t_i (ciclos):',
#                         continuous_update=False,
#                         style={'description_width': 'initial'}))

from ipywidgets import interact, FloatSlider, VBox, HBox, Button, Output
from IPython.display import display, clear_output

# Widget interactivo mejorado con controles
def crear_interfaz():
    """
    Crea interfaz interactiva con slider largo y botones de control
    """
    # Slider más largo
    slider_ti = FloatSlider(
        min=0.0, 
        max=1.0, 
        step=0.005,  # Paso más fino para mayor precisión
        value=0.0,
        description='$t_i$ (ciclos):',
        continuous_update=False,
        style={'description_width': '120px'},
        layout={'width': '800px'}  # Slider mucho más largo
    )
    
    # Botones de control
    btn_anterior = Button(
        description='◀ Paso atrás',
        button_style='info',
        tooltip='Retroceder 0.01 ciclos',
        layout={'width': '120px'}
    )
    
    btn_siguiente = Button(
        description='Paso adelante ▶',
        button_style='info',
        tooltip='Avanzar 0.01 ciclos',
        layout={'width': '120px'}
    )
    
    btn_reset = Button(
        description='↺ Reset',
        button_style='warning',
        tooltip='Volver a t_i = 0.25',
        layout={'width': '100px'}
    )
    
    btn_criticos = Button(
        description='⚡ Puntos críticos',
        button_style='success',
        tooltip='Ciclar entre t_i críticos',
        layout={'width': '140px'}
    )
    
    # Área de salida para el gráfico
    output = Output()
    
    # Funciones callback para los botones
    def paso_atras(b):
        slider_ti.value = max(slider_ti.min, slider_ti.value - 0.005)
    
    def paso_adelante(b):
        slider_ti.value = min(slider_ti.max, slider_ti.value + 0.005)
    
    def reset(b):
        slider_ti.value = 0.25
    
    # # Puntos críticos interesantes (máximos, mínimos, ceros del campo)
    # puntos_criticos = [0.0, 0.25, 0.5, 0.75, 1.0]
    # critico_actual = [0]  # Lista para mantener estado mutable
    
    # def siguiente_critico(b):
    #     critico_actual[0] = (critico_actual[0] + 1) % len(puntos_criticos)
    #     slider_ti.value = puntos_criticos[critico_actual[0]]
    
    # Conectar botones a funciones
    btn_anterior.on_click(paso_atras)
    btn_siguiente.on_click(paso_adelante)
    btn_reset.on_click(reset)
    #  btn_criticos.on_click(siguiente_critico)
    
    # Función que actualiza el gráfico
    def actualizar_grafico(change):
        with output:
            clear_output(wait=True)
            plot_trayectoria(slider_ti.value)
    
    # Observar cambios en el slider
    slider_ti.observe(actualizar_grafico, names='value')
    
    # Layout: botones en fila, slider debajo, gráfico al final
    # controles = HBox([btn_anterior, btn_siguiente, btn_reset, btn_criticos],
    #                  layout={'justify_content': 'center'})
    controles = HBox([btn_anterior, btn_siguiente, btn_reset],
                     layout={'justify_content': 'center'})
    
    interfaz = VBox([controles, slider_ti, output])
    
    # Mostrar interfaz y gráfico inicial
    display(interfaz)
    with output:
        plot_trayectoria(slider_ti.value)

# Ejecutar la interfaz
crear_interfaz()

1. Determina las ventanas de tiempos de ionización que dan lugar a trayectorias que dan lugar a recolisiones.

2. Considera la primera ventana de tiempo. Recorre la ventana de tiempo en pasos de 0.01 ciclos. Almacénalos en un array de numpy y imprime el contenido del array usando un bucle, de forma que se vea una tabla de los valotes de $t_i$, $t_{rec}$ y del tiempo de excursión $t_{exc}=t_{rec}-t_i$ y de la velocidad en el rescattering.
    - Determina la pareja $t_i$ y $t_{rec}$ de la trayectoria que da lugar a una recolisión con mayor velocidad. 
    - Cuál es el tiempo de excursión en este caso.
    - ¿Las trayectorias que se ionizan antes de ésta tienen tiempos de excursión más cortos o más largos?
    - Haz una figura de los tiempos de recolisión (eje izquierdo) y los tiempos de excursión (eje derecho) en función de $t_i$
    

3. Añade al array una nueva columna con la energía cinética, en unidades de $U_p$. Imprime de nuevo la tabla. ¿Cuál es la energía máxima de recolisión?